In [3]:
println("Hello")

Intitializing Scala interpreter ...

Spark Web UI available at http://103d525d728a:4041
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1753642250320)
SparkSession available as 'spark'


Hello


In [1]:
// Cell 1: Imports and Spark Session Setup
import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.expressions.Window
import java.time.{LocalDate, LocalDateTime}

Intitializing Scala interpreter ...

Spark Web UI available at http://9914a2c60656:4042
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1753703845536)
SparkSession available as 'spark'


import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.expressions.Window
import java.time.{LocalDate, LocalDateTime}


In [2]:
// Cell 2: Create Spark Session with Iceberg and MinIO Configuration

val spark = SparkSession.builder()
  .appName("Ascend_DPD_Processing")
  .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
  .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
  .config("spark.hadoop.fs.s3a.access.key", "admin")
  .config("spark.hadoop.fs.s3a.secret.key", "password")
  .config("spark.hadoop.fs.s3a.path.style.access", "true")
  .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
  .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
  .config("spark.sql.adaptive.enabled", "true")
  .config("spark.sql.adaptive.skewJoin.enabled", "true")
  .config("spark.sql.adaptive.localShuffleReader.enabled", "true")
  .config("spark.sql.optimizer.dynamicPartitionPruning.enabled", "true")
  .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
println("✅ Spark session created with Iceberg and MinIO configuration!")

✅ Spark session created with Iceberg and MinIO configuration!


spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@da6b1f7


In [1]:
val accountsDF = spark.table("ascenddb.dpd_data")
      .filter(col("CONS_ACCT_KEY") === 336)
      .orderBy(col("ACCT_DT"))
accountsDF.show()

Intitializing Scala interpreter ...

Spark Web UI available at http://9914a2c60656:4043
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1753708122605)
SparkSession available as 'spark'


+-------------+----------+---+
|CONS_ACCT_KEY|   ACCT_DT|DPD|
+-------------+----------+---+
|          336|1998-01-15| 37|
|          336|1998-02-15| 38|
|          336|1998-03-15| 39|
|          336|1998-05-15| 41|
|          336|1998-06-15| 42|
|          336|1998-07-15| 43|
|          336|1998-08-15| 44|
|          336|1998-09-15| 45|
|          336|1998-10-15| 46|
|          336|1998-11-15| 47|
|          336|1998-12-15| 48|
|          336|1999-01-15| 49|
|          336|1999-02-15| 50|
|          336|1999-03-15| 51|
|          336|1999-04-15| 52|
|          336|1999-05-15| 53|
|          336|1999-06-15| 54|
|          336|1999-07-15| 55|
|          336|1999-08-15| 56|
|          336|1999-09-15| 57|
+-------------+----------+---+
only showing top 20 rows



accountsDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CONS_ACCT_KEY: int, ACCT_DT: date ... 1 more field]


In [73]:
def generateDpdGrid(): Unit = {
  val startMonth = LocalDate.of(1998, 1, 1)
  val endMonth = LocalDate.of(1998, 2, 15)
  
  var currentMonth = startMonth
  
  while (!currentMonth.isAfter(endMonth)) {
    val nextMonth = currentMonth.plusMonths(1)
    
    println(s"Processing month: $currentMonth ...")
    
    val summaryColumnList = """CONS_ACCT_KEY, ACCT_DT, DPD""".replaceAll("\\s+", " ")
    
    // Load current month data
    val currentDf = spark.sql(s"""
      SELECT $summaryColumnList
      FROM ascenddb.dpd_data
      WHERE ACCT_DT >= DATE '${currentMonth.toString}' AND ACCT_DT < DATE '${nextMonth.toString}'
    """)

    // Load only latest record per account
    val latestPrev = spark.read.table("ascenddb.latest_dpd_summary")
      .withColumnRenamed("ACCT_DT", "ACCT_DT_prev")
    
    // Join datasets
    var mergedDf = currentDf.join(latestPrev, Seq("CONS_ACCT_KEY"), "left")

    // Calculate month difference and process DPD grid
    mergedDf = mergedDf.withColumn("MONTH_DIFF",
      (month(col("ACCT_DT")) - month(col("ACCT_DT_prev"))) +
      (year(col("ACCT_DT")) - year(col("ACCT_DT_prev"))) * 12
    ).withColumn("FILLER_ARRAY",
      when(col("MONTH_DIFF") > 1, 
        expr("transform(sequence(1, MONTH_DIFF - 1), x -> '?')")
      ).otherwise(array())
    ).withColumn("Merged_DPD_Array",
          concat(
            array(col("DPD")),
            col("FILLER_ARRAY"),
            when(col("DPD_GRID").isNotNull, split(col("DPD_GRID"), "~"))
              .otherwise(array())
          )
        )

    mergedDf = mergedDf.withColumn("array_size",
                        size(col("Merged_DPD_Array"))).withColumn(
                      "DPD_Array_Trimmed",
                      when(col("array_size") >= 36,
                        slice(col("Merged_DPD_Array"), 1, 36)
                      ).otherwise(
                        concat(
                          col("Merged_DPD_Array"),
                          expr("transform(sequence(1, 36 - array_size), x -> '?')")
                        )
                      )
                    ).withColumn("DPD_GRID",
                        concat_ws("~", slice(col("DPD_Array_Trimmed"), 1, 36))
                      )
                      .drop("array_size")

    // Save historical output
        val columnsToSelect = List(
          "CONS_ACCT_KEY", "ACCT_DT", "DPD_GRID", "DPD"
        )
        
        mergedDf.select(columnsToSelect.head, columnsToSelect.tail: _*)
          .write
          .format("iceberg")
          .mode("append")
          .saveAsTable("ascenddb.summary")
        
        // Load current month processed data
        val currentMonthDf = mergedDf.select("CONS_ACCT_KEY", "ACCT_DT", "DPD_GRID")
        
        // Revert name of ACCT_DT_prev
        val latestPrevRenamed = latestPrev.withColumnRenamed("ACCT_DT_prev", "ACCT_DT")
        
        // Merge: Take latest between old and new per account
        val windowSpec = Window.partitionBy("CONS_ACCT_KEY").orderBy(col("ACCT_DT").desc)
        
        val mergedLatest = latestPrevRenamed.union(currentMonthDf)
          .withColumn("rn", row_number().over(windowSpec))
          .filter(col("rn") === 1)
          .drop("rn")
        
        // Save merged latest back
        mergedLatest.write
          .mode("overwrite")
          .saveAsTable("ascenddb.latest_dpd_summary")
    
    // mergedDf.show(truncate=false)
    mergedDf.select(columnsToSelect.head, columnsToSelect.tail: _*).show()
    currentMonth = nextMonth
  }
}

generateDpdGrid: ()Unit


In [74]:
generateDpdGrid()

Processing month: 1998-01-01 ...
+-------------+----------+--------------------+---+
|CONS_ACCT_KEY|   ACCT_DT|            DPD_GRID|DPD|
+-------------+----------+--------------------+---+
|            1|1998-01-15|2~37~36~35~34~33~...|  2|
|            2|1998-01-15|3~38~37~36~35~34~...|  3|
|            3|1998-01-15|4~39~38~37~36~35~...|  4|
|            4|1998-01-15|5~40~39~38~37~36~...|  5|
|            5|1998-01-15|6~41~40~39~38~37~...|  6|
|            6|1998-01-15|7~42~41~40~39~38~...|  7|
|            7|1998-01-15|8~43~42~41~40~39~...|  8|
|            8|1998-01-15|9~44~43~42~41~40~...|  9|
|            9|1998-01-15|10~45~44~43~42~41...| 10|
|           10|1998-01-15|11~46~45~44~43~42...| 11|
|           11|1998-01-15|12~47~46~45~44~43...| 12|
|           12|1998-01-15|13~48~47~46~45~44...| 13|
|           13|1998-01-15|14~49~48~47~46~45...| 14|
|           14|1998-01-15|15~50~49~48~47~46...| 15|
|           15|1998-01-15|16~51~50~49~48~47...| 16|
|           16|1998-01-15|17~52

In [90]:
    val currentDf = spark.sql(s"""
      SELECT *
      FROM ascenddb.summary
      WHERE CONS_ACCT_KEY =336
    """).orderBy(col("ACCT_DT"))

    currentDf.show()

+-------------+----------+---+--------------------+
|CONS_ACCT_KEY|   ACCT_DT|DPD|            DPD_GRID|
+-------------+----------+---+--------------------+
|          336|1998-01-15| 37|37~?~?~?~?~?~?~?~...|
|          336|1998-02-15| 38|38~37~?~?~?~?~?~?...|
|          336|1998-03-15| 39|39~38~37~?~?~?~?~...|
|          336|1998-05-15| 41|41~?~39~38~37~?~?...|
|          336|1998-06-15| 42|42~41~?~39~38~37~...|
|          336|1998-07-15| 43|43~42~41~?~39~38~...|
|          336|1998-08-15| 44|44~43~42~41~?~39~...|
|          336|1998-09-15| 45|45~44~43~42~41~?~...|
|          336|1998-10-15| 46|46~45~44~43~42~41...|
|          336|1998-11-15| 47|47~46~45~44~43~42...|
|          336|1998-12-15| 48|48~47~46~45~44~43...|
|          336|1999-01-15| 49|49~48~47~46~45~44...|
|          336|1999-02-15| 50|50~49~48~47~46~45...|
|          336|1999-03-15| 51|51~50~49~48~47~46...|
|          336|1999-04-15| 52|52~51~50~49~48~47...|
|          336|1999-05-15| 53|53~52~51~50~49~48...|
|          3

currentDf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CONS_ACCT_KEY: int, ACCT_DT: date ... 2 more fields]


### Custom Optimizations I

In [84]:
val pad36 = Seq.fill(36)("?")

val trimAndPadUdf = udf((arr: Seq[String]) => {
  val padded = arr ++ pad36
  padded.take(36)
})

def generateDpdGrid(): Unit = {
  val startMonth = LocalDate.of(1998, 1, 1)
  val endMonth = LocalDate.of(2000, 12, 15)
  
  var currentMonth = startMonth
  
  while (!currentMonth.isAfter(endMonth)) {
    val nextMonth = currentMonth.plusMonths(1)
    
    println(s"Processing month: $currentMonth ...")
    
    val summaryColumnList = """CONS_ACCT_KEY, ACCT_DT, DPD"""
    
    // Load current month data
    val currentDf = spark.sql(s"""
      SELECT $summaryColumnList
      FROM ascenddb.dpd_data
      WHERE ACCT_DT >= DATE '${currentMonth.toString}' AND ACCT_DT < DATE '${nextMonth.toString}'
    """)

    // Load only latest record per account
    val latestPrev = spark.read.table("ascenddb.latest_dpd_summary")
      .withColumnRenamed("ACCT_DT", "ACCT_DT_prev")
    
    // Join datasets
    var mergedDf = currentDf.join(latestPrev, Seq("CONS_ACCT_KEY"), "left")

    // Calculate month difference and process DPD grid
    mergedDf = mergedDf.withColumn("MONTH_DIFF",
      months_between(col("ACCT_DT"), col("ACCT_DT_prev")).cast("int")
    ).withColumn("FILLER_ARRAY",
      when(col("MONTH_DIFF") > 1, 
        expr("transform(sequence(1, MONTH_DIFF - 1), x -> '?')")
      ).otherwise(array())
    ).withColumn("Merged_DPD_Array",
          concat(
            array(col("DPD")),
            col("FILLER_ARRAY"),
            when(col("DPD_GRID").isNotNull, split(col("DPD_GRID"), "~"))
              .otherwise(array())
          )
        )

    mergedDf = mergedDf.withColumn("DPD_Array_Trimmed", 
                        trimAndPadUdf(col("Merged_DPD_Array"))).withColumn("DPD_GRID",
                        concat_ws("~", slice(col("DPD_Array_Trimmed"), 1, 36))
                      )
                      

    // Save historical output
        val columnsToSelect = List(
          "CONS_ACCT_KEY", "ACCT_DT", "DPD_GRID", "DPD"
        )
        
        mergedDf.select(columnsToSelect.head, columnsToSelect.tail: _*)
          .write
          .format("iceberg")
          .mode("append")
          .saveAsTable("ascenddb.summary")
        
        // Load current month processed data
        val currentMonthDf = mergedDf.select("CONS_ACCT_KEY", "ACCT_DT", "DPD_GRID")
        
        // Revert name of ACCT_DT_prev
        val latestPrevRenamed = latestPrev.withColumnRenamed("ACCT_DT_prev", "ACCT_DT")
        
        // Merge: Take latest between old and new per account
        /* val windowSpec = Window.partitionBy("CONS_ACCT_KEY").orderBy(col("ACCT_DT").desc)
        
        val mergedLatest = latestPrevRenamed.union(currentMonthDf)
          .withColumn("rn", row_number().over(windowSpec))
          .filter(col("rn") === 1)
          .drop("rn")
          */

        val mergedLatest = latestPrevRenamed.union(currentMonthDf)
                                              .groupBy("CONS_ACCT_KEY")
                                              .agg(
                                                max_by(col("ACCT_DT"), col("ACCT_DT")).as("ACCT_DT"),
                                                max_by(col("DPD_GRID"), col("ACCT_DT")).as("DPD_GRID")
                                              )
                                                    
        // Save merged latest back
        mergedLatest.write
          .mode("overwrite")
          .saveAsTable("ascenddb.latest_dpd_summary")
    
    // mergedDf.show(truncate=false)
    // mergedDf.select(columnsToSelect.head, columnsToSelect.tail: _*).show(2)
    print("Done")
    currentMonth = nextMonth
  }
}

pad36: Seq[String] = List(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
trimAndPadUdf: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$6606/0x0000000842129040@6611a0bb,ArrayType(StringType,true),List(Some(class[value[0]: array<string>])),Some(class[value[0]: array<string>]),None,true,true)
generateDpdGrid: ()Unit


In [85]:
%%time
generateDpdGrid()

Processing month: 1998-01-01 ...
DoneProcessing month: 1998-02-01 ...
DoneProcessing month: 1998-03-01 ...
DoneProcessing month: 1998-04-01 ...
DoneProcessing month: 1998-05-01 ...
DoneProcessing month: 1998-06-01 ...
DoneProcessing month: 1998-07-01 ...
DoneProcessing month: 1998-08-01 ...
DoneProcessing month: 1998-09-01 ...
DoneProcessing month: 1998-10-01 ...
DoneProcessing month: 1998-11-01 ...
DoneProcessing month: 1998-12-01 ...
DoneProcessing month: 1999-01-01 ...
DoneProcessing month: 1999-02-01 ...
DoneProcessing month: 1999-03-01 ...
DoneProcessing month: 1999-04-01 ...
DoneProcessing month: 1999-05-01 ...
DoneProcessing month: 1999-06-01 ...
DoneProcessing month: 1999-07-01 ...
DoneProcessing month: 1999-08-01 ...
DoneProcessing month: 1999-09-01 ...
DoneProcessing month: 1999-10-01 ...
DoneProcessing month: 1999-11-01 ...
DoneProcessing month: 1999-12-01 ...
DoneProcessing month: 2000-01-01 ...
DoneProcessing month: 2000-02-01 ...
DoneProcessing month: 2000-03-01 ...
DoneP

## Optimization II

In [93]:
def generateDpdGrid(): Unit = {
  val startMonth = LocalDate.of(1998, 1, 1)
  val endMonth = LocalDate.of(2000, 12, 15)
  
  var currentMonth = startMonth
  
  while (!currentMonth.isAfter(endMonth)) {
    val nextMonth = currentMonth.plusMonths(1)
    
    println(s"Processing month: $currentMonth ...")
    
    val summaryColumnList = """CONS_ACCT_KEY, ACCT_DT, DPD"""
    
    // Load current month data
    /*
    val currentDf = spark.sql(s"""
      SELECT $summaryColumnList
      FROM ascenddb.dpd_data
      WHERE ACCT_DT >= DATE '${currentMonth.toString}' AND ACCT_DT < DATE '${nextMonth.toString}'
    """)
    */

    val currentDf = spark.table("ascenddb.dpd_data")
      .filter(col("ACCT_DT").between(lit(currentMonth), lit(nextMonth.minusDays(1))))
      .selectExpr("CONS_ACCT_KEY", "ACCT_DT", "DPD")

    // Load only latest record per account
    val latestPrev = spark.read.table("ascenddb.latest_dpd_summary")
      .withColumnRenamed("ACCT_DT", "ACCT_DT_prev")
    
    // Join datasets
    var mergedDf = currentDf.join(latestPrev, Seq("CONS_ACCT_KEY"), "left")

    // Calculate month difference and process DPD grid
    mergedDf = mergedDf.withColumn("MONTH_DIFF",
      months_between(col("ACCT_DT"), col("ACCT_DT_prev")).cast("int")
    ).withColumn("FILLER_ARRAY",
      when(col("MONTH_DIFF") > 1, 
        expr("transform(sequence(1, MONTH_DIFF - 1), x -> '?')")
      ).otherwise(array())
    ).withColumn("Merged_DPD_Array",
          concat(
            array(col("DPD")),
            col("FILLER_ARRAY"),
            when(col("DPD_GRID").isNotNull, split(col("DPD_GRID"), "~"))
              .otherwise(array())
          )
        )

    mergedDf = mergedDf.withColumn("DPD_Array_Trimmed",
                                      slice(
                                        concat(
                                          col("Merged_DPD_Array"),
                                          expr("array_repeat('?', 36)")
                                        ),
                                        1, 36
                                      )
                                    ).withColumn("DPD_GRID",
                        concat_ws("~", slice(col("DPD_Array_Trimmed"), 1, 36))
                      )
                      

    // Save historical output
        val columnsToSelect = List(
          "CONS_ACCT_KEY", "ACCT_DT", "DPD_GRID", "DPD"
        )
        
        mergedDf.select(columnsToSelect.head, columnsToSelect.tail: _*)
          .write
          .format("iceberg")
          .mode("append")
          .saveAsTable("ascenddb.summary")
        
        // Load current month processed data
        val currentMonthDf = mergedDf.select("CONS_ACCT_KEY", "ACCT_DT", "DPD_GRID")
        
        // Revert name of ACCT_DT_prev
        val latestPrevRenamed = latestPrev.withColumnRenamed("ACCT_DT_prev", "ACCT_DT")
        
        // Merge: Take latest between old and new per account
        /* val windowSpec = Window.partitionBy("CONS_ACCT_KEY").orderBy(col("ACCT_DT").desc)
        
        val mergedLatest = latestPrevRenamed.union(currentMonthDf)
          .withColumn("rn", row_number().over(windowSpec))
          .filter(col("rn") === 1)
          .drop("rn")
          */

        val mergedLatest = latestPrevRenamed.union(currentMonthDf)
                                              .groupBy("CONS_ACCT_KEY")
                                              .agg(
                                                max_by(col("ACCT_DT"), col("ACCT_DT")).as("ACCT_DT"),
                                                max_by(col("DPD_GRID"), col("ACCT_DT")).as("DPD_GRID")
                                              )
                                                    
        // Save merged latest back
        mergedLatest.write
          .mode("overwrite")
          .saveAsTable("ascenddb.latest_dpd_summary")
    
    // mergedDf.show(truncate=false)
    // mergedDf.select(columnsToSelect.head, columnsToSelect.tail: _*).show(2)
    print("Done")
    currentMonth = nextMonth
  }
}

generateDpdGrid: ()Unit


In [94]:
%%time
generateDpdGrid()

Processing month: 1998-01-01 ...
Time: 0.5732462406158447 seconds.



org.apache.spark.sql.catalyst.analysis.TableAlreadyExistsException:  [TABLE_OR_VIEW_ALREADY_EXISTS] Cannot create table or view `ascenddb`.`summary` because it already exists.